In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score


In [16]:
data = pd.read_csv('housing.csv')

In [25]:
data.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,20640.000000,20640.000000,20640.000000,20640.000000,20433.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,-119.569704,35.631861,28.639486,2635.763081,537.870553,1425.476744,499.539680,3.870671,206855.816909
std,2.003532,2.135952,12.585558,2181.615252,421.385070,1132.462122,382.329753,1.899822,115395.615874
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.800000,33.930000,18.000000,1447.750000,296.000000,787.000000,280.000000,2.563400,119600.000000
50%,-118.490000,34.260000,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800,179700.000000
75%,-118.010000,37.710000,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250,264725.000000
max,-114.310000,41.950000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [27]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [17]:
X = data.drop('median_house_value', axis=1)
y = data['median_house_value']

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [19]:
test_data = X_test.join(y_test)
train_data = X_train.join(y_train)

In [20]:
def prepering_data(train_data):
    
    imputer = SimpleImputer(strategy='most_frequent')
    OneHotEnc = OneHotEncoder(handle_unknown='ignore')
    
    train_data_filled = imputer.fit_transform(train_data)
    train_data_finall = pd.DataFrame(train_data_filled, columns=train_data.columns)
    
    cat_X = train_data_finall[['ocean_proximity']]
    cat_X_to_number = OneHotEnc.fit_transform(cat_X)
    
    cat_X_encoded_df = pd.DataFrame(cat_X_to_number.toarray(), columns=OneHotEnc.get_feature_names_out(['ocean_proximity']))
    
    train_data_finall = train_data_finall.drop(columns=['ocean_proximity'])
    finall_data = pd.concat([train_data_finall, cat_X_encoded_df], axis=1)
        
    return finall_data

In [21]:
train_data_finall = prepering_data(train_data)
test_data_finall = prepering_data(test_data)

In [29]:
train_data_finall.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity_<1H OCEAN,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
0,-121.62,36.67,31.0,2697.0,690.0,2220.0,665.0,2.5329,135200.0,1.0,0.0,0.0,0.0,0.0
1,-118.71,34.29,24.0,2983.0,406.0,1203.0,381.0,6.3236,302000.0,1.0,0.0,0.0,0.0,0.0
2,-117.69,34.13,8.0,2915.0,371.0,1271.0,354.0,7.9627,345400.0,0.0,1.0,0.0,0.0,0.0
3,-121.35,38.72,2.0,21897.0,3513.0,8652.0,2873.0,4.5432,151300.0,0.0,1.0,0.0,0.0,0.0
4,-120.97,37.66,19.0,1974.0,393.0,799.0,377.0,3.1286,137500.0,0.0,1.0,0.0,0.0,0.0


In [31]:
X_train = train_data_finall.drop(['median_house_value'], axis=1)
y_train = train_data_finall['median_house_value']

X_test = test_data_finall.drop(['median_house_value'], axis=1)
y_test = test_data_finall['median_house_value']

In [33]:
forest = RandomForestRegressor(n_estimators=100)
forest.fit(X_train, y_train)
forest.score(X_test, y_test)

0.8195608863302086